In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch_numopt
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset
from sklearn.datasets import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error

In [2]:
device = "cpu"

In [3]:
class Net(nn.Module):
    def __init__(self, input_size, device="cpu"):
        super().__init__()
        self.f1 = nn.Linear(input_size, 10, device=device)
        self.f2 = nn.Linear(10, 20, device=device)
        self.f3 = nn.Linear(20, 20, device=device)
        self.f4 = nn.Linear(20, 10, device=device)
        self.f5 = nn.Linear(10, 1, device=device)

        self.activation = nn.ReLU()
        # self.activation = nn.Sigmoid()

    def forward(self, x):
        x = self.activation(self.f1(x))
        x = self.activation(self.f2(x))
        x = self.activation(self.f3(x))
        x = self.activation(self.f4(x))
        x = self.f5(x)

        return x

In [4]:
# X, y = load_diabetes(return_X_y=True, scaled=False)
# X, y = make_regression(n_samples=1000, n_features=100)
X, y = make_friedman1(n_samples=1000, noise=1e-2)

X_scaler = MinMaxScaler()
X = X_scaler.fit_transform(X)
X = torch.Tensor(X).to(device)

y_scaler = MinMaxScaler()
y = y_scaler.fit_transform(y.reshape((-1, 1)))
y = torch.Tensor(y).to(device)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

torch_data = TensorDataset(X_train, y_train)
data_loader = DataLoader(torch_data, batch_size=1000)

torch.Size([800, 10]) torch.Size([800, 1])
torch.Size([200, 10]) torch.Size([200, 1])


# No Line Search

In [9]:
model = Net(input_size=X.shape[1], device=device)
loss_fn = nn.MSELoss()

opt = torch_numopt.NewtonRaphson(params=model.parameters(), model=model, lr=0.1, line_search_method="const")

all_loss = {}
for epoch in range(100):
    print("epoch: ", epoch, end="")
    all_loss[epoch + 1] = 0
    for batch_idx, (b_x, b_y) in enumerate(data_loader):
        pre = model(b_x)
        loss = loss_fn(pre, b_y)
        opt.zero_grad()
        loss.backward()

        # parameter update step based on optimizer
        opt.step(b_x, b_y, loss_fn)

        all_loss[epoch + 1] += loss
    all_loss[epoch + 1] /= len(data_loader)
    print(", loss: {}".format(all_loss[epoch + 1].detach().numpy().item()))

epoch:  0, loss: 0.4913996756076813
epoch:  1, loss: 12.60409927368164
epoch:  2, loss: 1.0642656087875366
epoch:  3, loss: 2.261833667755127
epoch:  4, loss: 0.15371383726596832
epoch:  5, loss: 0.04317309334874153
epoch:  6, loss: 0.017214877530932426
epoch:  7, loss: 0.012059775181114674
epoch:  8, loss: 0.008290204219520092
epoch:  9, loss: 0.006806775461882353
epoch:  10, loss: 0.005685069598257542
epoch:  11, loss: 0.0049519967287778854
epoch:  12, loss: 0.004418948199599981
epoch:  13, loss: 0.003955833613872528
epoch:  14, loss: 0.003595501882955432
epoch:  15, loss: 0.0032774542924016714
epoch:  16, loss: 0.0030173049308359623
epoch:  17, loss: 0.002818218432366848
epoch:  18, loss: 0.0026553079951554537
epoch:  19, loss: 0.0025193998590111732
epoch:  20, loss: 0.0023874593898653984
epoch:  21, loss: 0.0022924088407307863
epoch:  22, loss: 0.0021873945370316505
epoch:  23, loss: 0.0021038770209997892
epoch:  24, loss: 0.0020285965874791145
epoch:  25, loss: 0.00196572346612811

In [10]:
pred_train = model.forward(X_train).detach()
pred_test = model.forward(X_test).detach()
print(f"Train metrics: R2 = {r2_score(pred_train, y_train)}")
print(f"Test metrics:  R2 = {r2_score(pred_test, y_test)}")

Train metrics: R2 = 0.9804414733734099
Test metrics:  R2 = 0.9354575704083824


# Backtracking Line search

In [11]:
model = Net(input_size=X.shape[1], device=device)
loss_fn = nn.MSELoss()

opt = torch_numopt.NewtonRaphson(params=model.parameters(), model=model, lr=1, c1=1e-4, tau=0.1, line_search_method="backtrack", line_search_cond="armijo")
# opt = torch_numopt.NewtonRaphson(params=model.parameters(), model=model, lr=1, c1=1e-4, tau=0.5, line_search_method="backtrack", line_search_cond="wolfe")
# opt = torch_numopt.NewtonRaphson(params=model.parameters(), model=model, lr=1, c1=1e-4, tau=0.5, line_search_method="backtrack", line_search_cond="strong-wolfe")
# opt = torch_numopt.NewtonRaphson(params=model.parameters(), model=model, lr=1, c1=1e-4, tau=0.5, line_search_method="backtrack", line_search_cond="goldstein")

all_loss = {}
for epoch in range(100):
    print("epoch: ", epoch, end="")
    all_loss[epoch + 1] = 0
    for batch_idx, (b_x, b_y) in enumerate(data_loader):
        pre = model(b_x)
        loss = loss_fn(pre, b_y)
        opt.zero_grad()
        loss.backward()

        # parameter update step based on optimizer
        opt.step(b_x, b_y, loss_fn)

        all_loss[epoch + 1] += loss
    all_loss[epoch + 1] /= len(data_loader)
    print(", loss: {}".format(all_loss[epoch + 1].detach().numpy().item()))

epoch:  0, loss: 0.3968493640422821
epoch:  1, loss: 0.340989887714386
epoch:  2, loss: 0.26643890142440796
epoch:  3, loss: 0.2318674474954605
epoch:  4, loss: 0.18996557593345642
epoch:  5, loss: 0.1554243564605713
epoch:  6, loss: 0.06055226922035217
epoch:  7, loss: 0.03048025816679001
epoch:  8, loss: 0.025021716952323914
epoch:  9, loss: 0.008223396725952625
epoch:  10, loss: 0.0037970212288200855
epoch:  11, loss: 0.002330662216991186
epoch:  12, loss: 0.0017099428223446012
epoch:  13, loss: 0.0013892464339733124
epoch:  14, loss: 0.0011930714827030897
epoch:  15, loss: 0.001064147800207138
epoch:  16, loss: 0.0009652139851823449
epoch:  17, loss: 0.0008756962488405406
epoch:  18, loss: 0.0006279463996179402
epoch:  19, loss: 0.0005255608120933175
epoch:  20, loss: 0.00047610924229957163
epoch:  21, loss: 0.00044054267345927656
epoch:  22, loss: 0.00037718156818300486
epoch:  23, loss: 0.00031402334570884705
epoch:  24, loss: 0.0002850001910701394
epoch:  25, loss: 0.00026558089

In [12]:
pred_train = model.forward(X_train).detach()
pred_test = model.forward(X_test).detach()
print(f"Train metrics: R2 = {r2_score(pred_train, y_train)}")
print(f"Test metrics:  R2 = {r2_score(pred_test, y_test)}")

Train metrics: R2 = 0.9970758641683755
Test metrics:  R2 = 0.9850608917568883


# Bisection Line search

In [20]:
model = Net(input_size=X.shape[1], device=device)
loss_fn = nn.MSELoss()

# opt = torch_numopt.NewtonRaphson(params=model.parameters(), model=model, lr=1, c1=1e-4, tau=0.1, line_search_method="backtrack", line_search_cond="armijo")
# opt = torch_numopt.NewtonRaphson(params=model.parameters(), model=model, lr=1, c1=1e-4, tau=0.5, line_search_method="backtrack", line_search_cond="wolfe")
# opt = torch_numopt.NewtonRaphson(params=model.parameters(), model=model, lr=1, c1=1e-4, tau=0.5, line_search_method="backtrack", line_search_cond="strong-wolfe")
# opt = torch_numopt.NewtonRaphson(params=model.parameters(), model=model, lr=1, c1=1e-4, tau=0.5, line_search_method="backtrack", line_search_cond="goldstein")
opt = torch_numopt.NewtonRaphson(params=model.parameters(), model=model, lr=1, line_search_method="bisect")

all_loss = {}
for epoch in range(100):
    print("epoch: ", epoch, end="")
    all_loss[epoch + 1] = 0
    for batch_idx, (b_x, b_y) in enumerate(data_loader):
        pre = model(b_x)
        loss = loss_fn(pre, b_y)
        opt.zero_grad()
        loss.backward()

        # parameter update step based on optimizer
        opt.step(b_x, b_y, loss_fn)

        all_loss[epoch + 1] += loss
    all_loss[epoch + 1] /= len(data_loader)
    print(", loss: {}".format(all_loss[epoch + 1].detach().numpy().item()))

epoch:  0, loss: 0.24122892320156097
epoch:  1, loss: 0.06370922923088074
epoch:  2, loss: 0.05481192097067833
epoch:  3, loss: 0.03100678138434887
epoch:  4, loss: 0.020260978490114212
epoch:  5, loss: 0.015926534309983253
epoch:  6, loss: 0.010537773370742798
epoch:  7, loss: 0.008579334244132042
epoch:  8, loss: 0.0066173505038022995
epoch:  9, loss: 0.004721648525446653
epoch:  10, loss: 0.0036226711235940456
epoch:  11, loss: 0.0030223005451261997
epoch:  12, loss: 0.002677768236026168
epoch:  13, loss: 0.0023684590123593807
epoch:  14, loss: 0.002075226977467537
epoch:  15, loss: 0.001895594410598278
epoch:  16, loss: 0.001782854087650776
epoch:  17, loss: 0.0016814726404845715
epoch:  18, loss: 0.0016071621794253588
epoch:  19, loss: 0.001517284195870161
epoch:  20, loss: 0.0014546114252880216
epoch:  21, loss: 0.0013912329450249672
epoch:  22, loss: 0.0013421373441815376
epoch:  23, loss: 0.0012947340728715062
epoch:  24, loss: 0.0012718832585960627
epoch:  25, loss: 0.00122946

In [21]:
pred_train = model.forward(X_train).detach()
pred_test = model.forward(X_test).detach()
print(f"Train metrics: R2 = {r2_score(pred_train, y_train)}")
print(f"Test metrics:  R2 = {r2_score(pred_test, y_test)}")

Train metrics: R2 = 0.9791339730076761
Test metrics:  R2 = 0.8724100689172414
